In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

import sys, os
import torch
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm



#from human_body_prior.tools.omni_tools import copy2cpu as c2c

#os.environ['PYOPENGL_PLATFORM'] = 'egl'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# Choose the device to run the body model on.
comp_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
paths = []
folders = []
dataset_names = []
for root, dirs, files in os.walk('./inter_x'):
#     print(root, dirs, files)
#     for folder in dirs:
#         folders.append(os.path.join(root, folder))
    folders.append(root)
    for name in files:
        dataset_name = root.split('/')[2]
        if dataset_name not in dataset_names:
            dataset_names.append(dataset_name)
        paths.append(os.path.join(root, name))

In [4]:
save_root = './pose_data'
save_folders = [folder.replace('./inter_x', './pose_data') for folder in folders]
for folder in save_folders:
    os.makedirs(folder, exist_ok=True)
group_path = [[path for path in paths if name in path] for name in dataset_names]

In [5]:
group_path = group_path
all_count = sum([len(paths) for paths in group_path])
cur_count = 0

In [6]:
import numpy as np


from aitviewer.configuration import CONFIG as C
from aitviewer.renderables.meshes import Meshes
from aitviewer.renderables.skeletons import Skeletons
from aitviewer.renderables.plane import Plane
from aitviewer.viewer import Viewer
from scipy.spatial.transform import Rotation as R

from aitviewer.models.smpl import SMPLLayer
from aitviewer.renderables.smpl import SMPLSequence

ex_fps = 20
def interx_to_pose(src_path, save_path):
    bdata = np.load(src_path, allow_pickle=True)
    fps = 120 #原始帧率

    down_sample = int(fps / ex_fps)

    nf = bdata['pose_body'][::down_sample,...].shape[0] #138

    betas = bdata['betas'] #[1,10]
    poses_root = bdata['root_orient'][::down_sample,...] #[138,3]
    poses_body = bdata['pose_body'][::down_sample,...].reshape(nf,-1) #[138,63]
    poses_lhand = bdata['pose_lhand'][::down_sample,...].reshape(nf,-1) #[138,45]
    poses_rhand = bdata['pose_rhand'][::down_sample,...].reshape(nf,-1) #[138,45]
    transl = bdata['trans'][::down_sample,...] #[138,3]
    gender = str(bdata['gender']) #male


    # create body models
    smplx_layer = SMPLLayer(model_type='smplx',gender=gender,num_betas=10,device=C.device)
    smplx_seq = SMPLSequence(poses_body=poses_body,
                        smpl_layer=smplx_layer,
                        poses_root=poses_root,
                        betas=betas,
                        trans=transl,
                        poses_left_hand=poses_lhand,
                        poses_right_hand=poses_rhand,
                        device=C.device,
                        color=(0.11, 0.53, 0.8, 1.0)
                        )



    pose_seq_np = smplx_seq.joints
    directory, filename = os.path.split(save_path)
    new_filename = os.path.basename(directory) + filename
    directory = './joints/'
    new_save_path = os.path.join(directory, new_filename)
    np.save(new_save_path, pose_seq_np)
    return fps

This will take a few hours for all datasets, here we take one dataset as an example

To accelerate the process, you could run multiple scripts like this at one time.

In [9]:
import time
for paths in group_path:
    dataset_name = paths[0].split('/')[2]
#   pbar = tqdm(paths)
#    pbar.set_description('Processing: %s'%dataset_name)
    fps = 0
    for path in pbar:
        save_path = path.replace('./inter_x', './pose_data')
        save_path = save_path[:-3] + 'npy'
        # fps = amass_to_pose(path, save_path)
        fps = interx_to_pose(path,save_path)
        
    cur_count += len(paths)
#  print('Processed / All (fps %d): %d/%d'% (fps, cur_count, all_count) )
    time.sleep(0.5)
print(cur_count)
print(all_count)

22910
22776


: 